In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!git clone --branch epochs20 https://github.com/Anjaas85/authentic_vs_AI_generated_photos


Cloning into 'authentic_vs_AI_generated_photos'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 75 (delta 4), reused 7 (delta 1), pack-reused 62 (from 1)
Receiving objects: 100% (75/75), 191.34 MiB | 12.61 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [2]:
%cd authentic_vs_AI_generated_photos

/content/authentic_vs_AI_generated_photos


In [3]:
!ls


basic_graphics.ipynb  data  metrics  training20ep.ipynb		     utils
checkpoints	      logs  models   two_class_classification.ipynb


In [5]:
!git branch new_branch

In [7]:
!git checkout new_branch

Already on 'new_branch'


In [9]:
# Downloading and Distributing the dataset
from torchvision.datasets import CIFAR10
from sklearn.model_selection import train_test_split
import os
from PIL import Image

# Step 1: Define paths
data_root = "./data"
train_dir = os.path.join(data_root, "train")
val_dir = os.path.join(data_root, "val")
class_dirs = ["class_0", "class_1"]

# Step 2: Create folder structure
for directory in [train_dir, val_dir]:
    for class_dir in class_dirs:
        os.makedirs(os.path.join(directory, class_dir), exist_ok=True)

# Step 3: Download CIFAR10 dataset
cifar10 = CIFAR10(root="./", download=True)
data, labels = cifar10.data, cifar10.targets

# Step 4: Simulate CIFAKE10 (first 5 classes as "authentic", next 5 as "AI-generated")
labels = [0 if label < 5 else 1 for label in labels]

# Step 5: Split data into train and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(
    data, labels, test_size=0.2, random_state=42, stratify=labels
)

# Helper function to save images
def save_images(images, labels, save_dir):
    for idx, (image, label) in enumerate(zip(images, labels)):
        class_folder = os.path.join(save_dir, f"class_{label}")
        img = Image.fromarray(image)
        img.save(os.path.join(class_folder, f"{idx}.png"))

# Step 6: Save images to train and val folders
save_images(train_data, train_labels, train_dir)
save_images(val_data, val_labels, val_dir)

print("CIFAKE10 dataset distributed into train and validation folders.")




from models.densenet_model import create_densenet
from utils.data_loader import create_dataloaders, create_test_loader
from utils.evaluation import evaluate_model
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_recall_curve, f1_score
import numpy as np
import json
from sklearn.metrics import confusion_matrix, precision_recall_curve, f1_score
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Paths and parameters
data_dir = 'data'

# Load train, validation, and test data
dataloaders = create_dataloaders(data_dir, batch_size=32)
test_loader = create_test_loader( batch_size=32)

# Load model
model = create_densenet(num_classes=2).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay = 1e-4)

#learnin rate
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.5, patience = 3, verbose = True)

"""
def train_model(model, criterion, optimizer, dataloaders, num_epochs=20, device='cpu'):
    # Set up folders
    metrics_file_path = os.path.join('metrics', 'metrics_by_epoch.txt')
    os.makedirs('metrics', exist_ok=True)
    os.makedirs('checkpoints', exist_ok=True)

    # Initialize storage for metrics
    metrics_data = {
        "precision_recall": [],
        "f1_scores": [],
        "val_probs": [],
        "val_labels": [],
        "confusion_matrices": [],
        "epoch_metrics": []
    }

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

    with open(metrics_file_path, 'w') as f:
        f.write(f'Epoch\tPhase\tLoss\tAccuracy\n')

        for epoch in range(num_epochs):
            print(f'Epoch {epoch+1}/{num_epochs}')
            print('-' * 10)

            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()
                else:
                    model.eval()

                running_loss = 0.0
                running_corrects = 0
                all_labels = []
                all_probs = []

                for inputs, labels in dataloaders[phase]:
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)
                        probs = torch.softmax(outputs, dim=1)[:, 1]

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    all_labels.extend(labels.cpu().numpy().tolist())
                    all_probs.extend(probs.detach().cpu().numpy().tolist())

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
                f.write(f'{epoch+1}\t{phase}\t{epoch_loss:.4f}\t{epoch_acc:.4f}\n')

                if phase == 'val':
                    scheduler.step(epoch_loss)
                    precision, recall, thresholds = precision_recall_curve(all_labels, all_probs)
                    f1 = f1_score(all_labels, np.round(all_probs))
                    confusion_mat = confusion_matrix(all_labels, np.round(all_probs))

                    metrics_data["precision_recall"].append({
                        "precision": precision.tolist(),
                        "recall": recall.tolist(),
                        "thresholds": thresholds.tolist(),
                    })
                    metrics_data["f1_scores"].append(float(f1))
                    metrics_data["val_probs"].append(all_probs)
                    metrics_data["val_labels"].append(all_labels)
                    metrics_data["confusion_matrices"].append(confusion_mat.tolist())
                    metrics_data["epoch_metrics"].append({
                        "epoch": epoch+1,
                        "loss": float(epoch_loss),
                        "accuracy": float(epoch_acc)
                    })

    # Save metrics data as JSON
    with open(os.path.join('metrics', 'metrics_data.json'), 'w') as json_file:
        json.dump(metrics_data, json_file, indent=4)

    # Save final model
    model_save_path = os.path.join('checkpoints', 'densenet_ai_vs_authentic.pth')
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved at {model_save_path}")

    print(f"Metrics saved in {metrics_file_path} and metrics_data.json")
    return model

"""

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import precision_recall_curve, f1_score, confusion_matrix
import numpy as np
import json
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

# Improved training loop with GPU support and metric saving
def train_model(model, criterion, optimizer, dataloaders, num_epochs=20, device='cuda'):
    # Set up folders for saving metrics and model
    metrics_file_path = os.path.join('metrics', 'metrics_by_epoch.txt')
    os.makedirs('metrics', exist_ok=True)
    os.makedirs('checkpoints', exist_ok=True)

    # Initialize storage for metrics
    metrics_data = {
        "precision_recall": [],
        "f1_scores": [],
        "val_probs": [],
        "val_labels": [],
        "confusion_matrices": [],
        "epoch_metrics": []
    }

    # Learning rate scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

    with open(metrics_file_path, 'w') as f:
        f.write(f'Epoch\tPhase\tLoss\tAccuracy\n')

        for epoch in range(num_epochs):
            print(f'Epoch {epoch+1}/{num_epochs}')
            print('-' * 10)

            # Training and Validation phases
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()  # Set model to evaluation mode

                running_loss = 0.0
                running_corrects = 0
                all_labels = []
                all_probs = []

                # Iterate through batches in dataloader
                for inputs, labels in dataloaders[phase]:
                    # Move data to the correct device (GPU)
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        # Forward pass
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)
                        probs = torch.softmax(outputs, dim=1)[:, 1]

                        if phase == 'train':
                            loss.backward()  # Backpropagation
                            optimizer.step()  # Update weights

                    # Update running loss and correct predictions
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                    all_labels.extend(labels.cpu().numpy().tolist())
                    all_probs.extend(probs.detach().cpu().numpy().tolist())

                # Calculate and log epoch statistics
                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
                f.write(f'{epoch+1}\t{phase}\t{epoch_loss:.4f}\t{epoch_acc:.4f}\n')

                if phase == 'val':
                    # Step the scheduler with validation loss
                    scheduler.step(epoch_loss)

                    # Calculate additional metrics for evaluation
                    precision, recall, thresholds = precision_recall_curve(all_labels, all_probs)
                    f1 = f1_score(all_labels, np.round(all_probs))
                    confusion_mat = confusion_matrix(all_labels, np.round(all_probs))

                    metrics_data["precision_recall"].append({
                        "precision": precision.tolist(),
                        "recall": recall.tolist(),
                        "thresholds": thresholds.tolist(),
                    })
                    metrics_data["f1_scores"].append(float(f1))
                    metrics_data["val_probs"].append(all_probs)
                    metrics_data["val_labels"].append(all_labels)
                    metrics_data["confusion_matrices"].append(confusion_mat.tolist())
                    metrics_data["epoch_metrics"].append({
                        "epoch": epoch+1,
                        "loss": float(epoch_loss),
                        "accuracy": float(epoch_acc)
                    })

    # Save metrics data as JSON
    with open(os.path.join('metrics', 'metrics_data.json'), 'w') as json_file:
        json.dump(metrics_data, json_file, indent=4)

    # Save final trained model
    model_save_path = os.path.join('checkpoints', 'densenet_ai_vs_authentic.pth')
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved at {model_save_path}")

    print(f"Metrics saved in {metrics_file_path} and metrics_data.json")
    return model



# Train the model
trained_model = train_model(model, criterion, optimizer, dataloaders, num_epochs=10)



# Save the model
torch.save(trained_model.state_dict(), 'checkpoints/densenet_ai_vs_authentic.pth')
print("Model trained and saved!")

# Evaluate the model
print("Evaluating on validation set:")
evaluate_model(trained_model, dataloaders['val'], device,"validation")

print("Evaluating on test set:")
evaluate_model(trained_model, test_loader, device, "test")




Files already downloaded and verified
CIFAKE10 dataset distributed into train and validation folders.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10
----------


...

train Loss: 0.3246 Acc: 0.8578
val Loss: 0.2841 Acc: 0.8756
Model saved at checkpoints/densenet_ai_vs_authentic.pth
Metrics saved in metrics/metrics_by_epoch.txt and metrics_data.json
Model trained and saved!
Evaluating on validation set:


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.91      0.88      5000
           1       0.90      0.85      0.87      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

Confusion Matrix:
[[4526  474]
 [ 770 4230]]
Evaluating on test set:


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.90      0.88      5000
           1       0.89      0.85      0.87      5000

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000

Confusion Matrix:
[[4503  497]
 [ 768 4232]]


In [ ]:
!git config --global user.email "anja.stanic2001@gmail.com"
!git config --global user.name "Anja Stanic"

In [ ]:
!git commit -m "First results for initial training on test and val"

In [ ]:
!git reset --soft HEAD~1


In [ ]:
!git reset

In [ ]:
!git log

In [ ]:
!git add new_main.ipynb

In [ ]:
!git add checkpoints/densenet_ai_vs_authentic.pth

In [ ]:
!git add utils/data_loader.py
!git add utils/evaluation.py
!git add models/densenet_model.py

In [11]:
!git status

On branch new_branch
Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    basic_graphics.ipynb
	modified:   checkpoints/densenet_ai_vs_authentic.pth
	modified:   metrics/metrics.txt
	modified:   metrics/metrics_by_epoch.txt
	modified:   metrics/metrics_data.json
	modified:   models/densenet_model.py
	deleted:    two_class_classification.ipynb
	modified:   utils/data_loader.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	cifar-10-batches-py/
	cifar-10-python.tar.gz
	data/train/class_0/
	data/train/class_1/
	data/val/class_0/
	data/val/class_1/
	models/__pycache__/
	utils/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git commit -m "Result of inital training for val and test"

[main 7d673d2d] Result of inital training for val and test
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 checkpoints/densenet_ai_vs_authentic.pth


In [ ]:
!